Mounting Google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Load the dataset

In [2]:
dataset_location = "/content/drive/MyDrive/IBM/spam.csv"

#Import Required Library

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

#Read dataset and do pre-processsing

In [9]:
df = pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
#dropping unwanted columns
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [11]:
# Count of Spam and Ham values
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [12]:
# Label Encoding target column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [14]:
# Test and train split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [15]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

#Create Model and Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [16]:
# Creating LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

#Compile the model

In [18]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                           

#Fit the model

In [19]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=20,
          validation_split=0.2)

Epoch 1/20
30/30 [==============================] - 10s 252ms/step - loss: 0.3232 - accuracy: 0.8741 - val_loss: 0.1792 - val_accuracy: 0.9283
Epoch 2/20
30/30 [==============================] - 7s 232ms/step - loss: 0.0903 - accuracy: 0.9781 - val_loss: 0.0612 - val_accuracy: 0.9842
Epoch 3/20
30/30 [==============================] - 9s 299ms/step - loss: 0.0423 - accuracy: 0.9892 - val_loss: 0.0487 - val_accuracy: 0.9873
Epoch 4/20
30/30 [==============================] - 7s 233ms/step - loss: 0.0319 - accuracy: 0.9902 - val_loss: 0.0808 - val_accuracy: 0.9768
Epoch 5/20
30/30 [==============================] - 7s 233ms/step - loss: 0.0236 - accuracy: 0.9937 - val_loss: 0.0510 - val_accuracy: 0.9842
Epoch 6/20
30/30 [==============================] - 7s 233ms/step - loss: 0.0181 - accuracy: 0.9952 - val_loss: 0.0559 - val_accuracy: 0.9852
Epoch 7/20
30/30 [==============================] - 7s 233ms/step - loss: 0.0136 - accuracy: 0.9966 - val_loss: 0.0720 - val_accuracy: 0.9789
Epoch

#Save the model

In [20]:
model.save("IBM_model.h1")

#Test the model

In [22]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [23]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 19ms/step - loss: 0.2457 - accuracy: 0.9701
Accuracy: 0.970


In [24]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 20ms/step
[[1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.024]
 [1.   ]
 [1.   ]
 [0.   ]]


In [26]:
print(Y_test[25:40])

[[1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]]
